In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, UpSampling2D, BatchNormalization, Activation
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from skimage.color import rgb2lab, lab2rgb
import matplotlib.pyplot as plt
# import numpy as np
from tensorflow.keras.datasets import cifar10

In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()


170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 12s 0us/step


In [3]:
# Set a fraction or fixed number of samples to use
fraction = 0.1  # Use 10% of the data
num_train_samples = int(len(x_train) * fraction)
num_test_samples = int(len(x_test) * fraction)

In [4]:
# Randomly select a subset of training and testing data
indices_train = np.random.choice(len(x_train), num_train_samples, replace=False)
indices_test = np.random.choice(len(x_test), num_test_samples, replace=False)

x_train_reduced, y_train_reduced = x_train[indices_train], y_train[indices_train]
x_test_reduced, y_test_reduced = x_test[indices_test], y_test[indices_test]


In [5]:
# Print new dataset sizes
print(f"Reduced training set size: {x_train_reduced.shape}")
print(f"Reduced testing set size: {x_test_reduced.shape}")


Reduced training set size: (5000, 32, 32, 3)
Reduced testing set size: (1000, 32, 32, 3)


In [6]:
# Normalize images to [0, 1] range
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0


NameError: name 'X_train' is not defined

In [7]:
# Convert RGB to Lab color space
def rgb_to_lab(images):
    return np.array([rgb2lab(img) for img in images])

def preprocess_data(images):
    lab_images = rgb_to_lab(images)
    L = lab_images[..., 0] / 100.0  # Normalize L channel to [0, 1]
    ab = lab_images[..., 1:] / 128.0  # Normalize a and b channels to [-1, 1]
    return L[..., np.newaxis], ab

L_train, ab_train = preprocess_data(X_train)
L_test, ab_test = preprocess_data(X_test)


NameError: name 'X_train' is not defined

In [ ]:
# Split training data for validation
L_train, L_val, ab_train, ab_val = train_test_split(L_train, ab_train, test_size=0.1, random_state=42)


In [ ]:
# Model architecture
def build_colorization_model():
    inputs = Input(shape=(32, 32, 1))

In [ ]:
 # Encoder
    x = Conv2D(64, (3, 3), padding='same', strides=1)(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(128, (3, 3), padding='same', strides=2)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(256, (3, 3), padding='same', strides=2)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

In [ ]:
    # Decoder
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(128, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(64, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    outputs = Conv2D(2, (3, 3), padding='same', activation='tanh')(x)  # Output ab channels

    model = Model(inputs, outputs)
    return model

model = build_colorization_model()
model.compile(optimizer='adam', loss='mse', metrics=['mae']

In [ ]:
# Training
history = model.fit(
    L_train, ab_train,
    validation_data=(L_val, ab_val),
    epochs=20,
    batch_size=64
)

In [ ]:
# Save the model in HDF5 format
model.save("colorization_model.h5")
print("Model saved as colorization_model.h5")